# 3D Structure

In [1]:
import glob
import os
import re
import subprocess
import urllib2

import cdpybio as cpb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import pybedtools as pbt
import seaborn as sns
import vcf as pyvcf

import cardipspy as cpy
import ciepy

%matplotlib inline

import socket
if socket.gethostname() == 'fl-hn1' or socket.gethostname() == 'fl-hn2':
    pbt.set_tempdir('/frazer01/home/cdeboever/tmp')
    
outdir = os.path.join(ciepy.root, 'output',
                      '3d_structure')
cpy.makedir(outdir)

private_outdir = os.path.join(ciepy.root, 'private_output',
                              '3d_structure')
cpy.makedir(private_outdir)

/frazer01/home/cdeboever/software/anaconda/envs/cie/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [12]:
tg = pd.read_table(cpy.gencode_transcript_gene, index_col=0, 
                   header=None, squeeze=True)
gene_info = pd.read_table(cpy.gencode_gene_info, index_col=0)
fn = os.path.join(ciepy.root, 'output', 'eqtl_input', 
                  'tpm_log_filtered_phe_std_norm_peer_resid.tsv')
exp = pd.read_table(fn, index_col=0)

fn = os.path.join(ciepy.root, 'output', 'eqtl_processing', 'qvalues.tsv')
qvalues = pd.read_table(fn, index_col=0)
qvalues.columns = ['{}_gene'.format(x) for x in qvalues.columns]
fn = os.path.join(ciepy.root, 'output', 'eqtl_processing', 'most_sig_single.tsv')
most_sig = pd.read_table(fn, index_col=0)

genes = pbt.BedTool(cpy.gencode_gene_bed)

fn = os.path.join(os.path.split(cpy.roadmap_15_state_annotation)[0], 'EIDlegend.txt')
roadmap_ids = pd.read_table(fn, squeeze=True, index_col=0, header=None)

fn = os.path.join(ciepy.root, 'output', 'eqtl_processing', 'gene_variant_pairs.tsv')
sig_variants = pd.read_table(fn)

## TADs

I'd like to look for enrichment of eQTLs in TADs. As in the Grubert et al. 2015 paper, I
can mirror the position of the variant over the gene and see whether they fall within the
same TAD as often. They also shuffled the locations of the TADs and compared the number of
distal (> 50 kb) QTLs falling within the same TAD as the gene against the shuffled data.
I can do the same analysis using sets of null variants as well.

In [3]:
# Download TADs from Dixon et al. 2012. These coordinates are in hg18 so we'll download the
# liftOver chain files and convert the bed file.

dy = os.path.join(private_outdir, 'hESC')
if not os.path.exists(dy):
    out = os.path.join(private_outdir, 'hESC.domain.tar.gz')
    !curl http://132.239.201.216/mouse/hi-c/hESC.domain.tar.gz > {out}
    !tar -C {private_outdir} -xvf {out}
    !rm {out}
    
fn = os.path.join(private_outdir, 'hg18ToHg19.over.chain')
if not os.path.exists(fn):
    url = ('http://hgdownload.cse.ucsc.edu/goldenPath/hg18/liftOver/hg18ToHg19.over.chain.gz')
    !curl {url} | zcat > {fn}

In [9]:
# Read in TADs, convert to hg19.
tads = pbt.BedTool(os.path.join(private_outdir, 'hESC', 'combined', 'total.combined.domain'))
n = len(tads)
mapped = os.path.join(outdir, 'hg19_hESC_tads_mapped.bed')
unmapped = os.path.join(outdir, 'hg19_hESC_tads_unmapped.txt')
tads = cpb.analysis.liftover_bed(tads, os.path.join(private_outdir, 'hg18ToHg19.over.chain'),
                                 mapped, unmapped)
print('Converted {} of {} TADs to hg19.'.format(len(tads), n))

Converted 3062 of 3127 TADs to hg19.


In [14]:
sig = most_sig[most_sig.sig_gene]

In [16]:
sig.head()

,chrom,start,end,loc,marker_id,ns,ac,callrate,genocnt,maf,...,beta,sebeta,r2,ref,alt,length,vtype,pvalue_gene,qvalue_gene,sig_gene
ENSG00000000460.12,chr1,169747782,169747783,chr1:169747783,1:169747783_T/G,207,29,1,182/21/4,0.07005,...,0.8657,0.16920,0.1137,T,G,0,snp,0.0007,0.001608,True
ENSG00000001561.6,chr6,46112388,46112389,chr6:46112389,6:46112389_A/G_rs1048077,207,279,1,26/83/98,0.32609,...,0.7554,0.08975,0.2577,A,G,0,snp,0.0001,0.000284,True
ENSG00000002016.12,chr12,1043833,1043834,chr12:1043834,12:1043834_T/C_rs10849594,207,59,1,153/49/5,0.14251,...,-0.8211,0.13120,0.1611,T,C,0,snp,0.0001,0.000284,True
ENSG00000003056.3,chr12,9095748,9095749,chr12:9095749,12:9095749_T/C_rs7309922,207,189,1,69/87/51,0.45652,...,-0.6825,0.08893,0.2240,T,C,0,snp,0.0001,0.000284,True
ENSG00000003147.13,chr7,9247738,9247739,chr7:9247739,7:9247739_A/G_rs12537168,207,125,1,100/89/18,0.30193,...,-0.5109,0.09957,0.1143,A,G,0,snp,0.0012,0.002603,True
